# 数据处理

## 数据处理 - 健康

NDB_H28.xlsx


> 健康数据，NDB

>     col 0	县
>     col 1	指标，取BMI，腹围，空腹血糖三项，代表了肥胖和糖尿病
>     col 9	男性平均
>     col 17	女性平均

In [135]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度
pd.set_option('display.max_rows', 25) # 预览时最多显示行数
pd.set_option('display.float_format', lambda x : '%.2f' % x)  # 不使用科学计数法

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns

def explore(df, n=5, describe=False, info=False):
  print('rows x cols:', df.shape)
  print('column name:', '\t'.join(df.columns))
  print()
  if describe:
    print('==== describe ====')
    print(df.describe())
  if info:
    print('==== info ====')
    print(df.info())
  print('==== sample ====')
  return df.sample(n=n, random_state=100)

# explore_df(df)

def find_in(df, col, pat):
  ''' 在 col 中 查找命中 pat 的数据
      如果 pat 是 字符串, 视为模糊查找 
      如果 pat 是 list, 或者非字符串型, 视为精确查找 '''
  if isinstance(pat, str):
    return df[df[col].str.contains(pat)]
  if not isinstance(pat, (list, set)):
    pat = [pat]
  return df[df[col].isin(pat)]


### 预览 NDB_H28.xlsx

In [136]:
health_df = pd.read_excel("../NDB_H28.xlsx", header=1, index_col='都道府県')
health_df.reset_index(inplace=True)

In [137]:
health_df.head(18)

,都道府県,検査項目,全体,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,男,NaN,NaN,NaN,NaN,NaN,NaN,NaN,女,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,40～44歳,45～49歳,50～54歳,55～59歳,60～64歳,65～69歳,70～74歳,中計,40～44歳,45～49歳,50～54歳,55～59歳,60～64歳,65～69歳,70～74歳,中計
2,NaN,NaN,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均,平均
3,北海道,BMI[kg/㎡],24.39,24.52,24.41,24.28,24.15,23.98,23.82,24.28,22.04,22.31,22.26,22.39,22.65,22.77,22.99,22.47
4,NaN,腹囲[cm],84.49,85.33,85.54,85.56,85.54,85.35,84.93,85.24,76.81,77.81,78.22,79.08,80.12,80.51,81.36,79.04
5,NaN,空腹時血糖[mg/dl],95.36,98.12,101.15,103.51,105.07,104.86,103.83,100.96,89.24,90.46,91.96,93.95,94.83,95.59,96.27,92.88
6,NaN,HbA1c（NGSP）[％],5.46,5.55,5.64,5.72,5.79,5.79,5.77,5.66,5.32,5.38,5.49,5.59,5.63,5.66,5.69,5.54
7,NaN,収縮期血圧[mmHg],121.13,123.21,125.53,128.17,130.54,132.46,133.58,126.79,112.43,115.63,118.60,121.39,125.18,128.65,131.32,121.52
8,NaN,拡張期血圧[mmHg],75.93,78.19,79.94,80.59,80.04,78.91,77.03,78.67,69.08,70.92,72.74,73.79,74.44,74.89,74.57,72.81
9,NaN,中性脂肪[mg/dl],135.33,140.34,142.30,138.24,133.23,127.01,121.26,135.48,78.66,84.12,92.89,98.45,102.74,105.33,105.98,94.91


In [138]:
# `中計` `中計.1` 分别是男女平均, 在`Unnamed: 9` `Unnamed: 17` 的位置

health_df = health_df[['都道府県', '検査項目', 'Unnamed: 9', 'Unnamed: 17']]
health_df.columns = ['都道府県', '検査項目', '男性平均', '女性平均']

In [139]:
health_df.head(25)

,都道府県,検査項目,男性平均,女性平均
0,NaN,NaN,NaN,NaN
1,NaN,NaN,中計,中計
2,NaN,NaN,平均,平均
3,北海道,BMI[kg/㎡],24.28,22.47
4,NaN,腹囲[cm],85.24,79.04
5,NaN,空腹時血糖[mg/dl],100.96,92.88
6,NaN,HbA1c（NGSP）[％],5.66,5.54
7,NaN,収縮期血圧[mmHg],126.79,121.52
8,NaN,拡張期血圧[mmHg],78.67,72.81
9,NaN,中性脂肪[mg/dl],135.48,94.91


In [140]:
# 从检查项目中, 保留 `BMI[kg/㎡]` `腹囲[cm]` `空腹時血糖[mg/dl]`
# 其余去掉去掉第一行是记录 `平均` 的表头, 去掉

health_df = health_df[health_df.検査項目.isin(['BMI[kg/㎡]', '腹囲[cm]', '空腹時血糖[mg/dl]'])]

In [141]:
health_df = health_df.reset_index()
health_df.drop(columns=['index'], inplace=True)
health_df
# 总计 144 行, 
# 144 / 3 = 48, 因为最后有一个 "全国"

,都道府県,検査項目,男性平均,女性平均
0,北海道,BMI[kg/㎡],24.28,22.47
1,NaN,腹囲[cm],85.24,79.04
2,NaN,空腹時血糖[mg/dl],100.96,92.88
3,青森県,BMI[kg/㎡],24.13,22.83
4,NaN,腹囲[cm],84.43,79.94
5,NaN,空腹時血糖[mg/dl],104.20,95.67
6,岩手県,BMI[kg/㎡],24.07,22.80
7,NaN,腹囲[cm],84.73,80.70
8,NaN,空腹時血糖[mg/dl],102.76,94.59
9,宮城県,BMI[kg/㎡],24.12,22.59


In [142]:
# 去掉 "全国" 的三行
health_df = health_df.drop([141, 142, 143])

In [143]:
# 将 `都道府県` 重复三次

series = []
for name in health_df.都道府県:
  if isinstance(name, str):
    series.append(name)
  else:
    series.append(series[-1])  # 检测到 NaN 就重复上一次的地名

health_df['都道府県'] = series
health_df

,都道府県,検査項目,男性平均,女性平均
0,北海道,BMI[kg/㎡],24.28,22.47
1,北海道,腹囲[cm],85.24,79.04
2,北海道,空腹時血糖[mg/dl],100.96,92.88
3,青森県,BMI[kg/㎡],24.13,22.83
4,青森県,腹囲[cm],84.43,79.94
5,青森県,空腹時血糖[mg/dl],104.20,95.67
6,岩手県,BMI[kg/㎡],24.07,22.80
7,岩手県,腹囲[cm],84.73,80.70
8,岩手県,空腹時血糖[mg/dl],102.76,94.59
9,宮城県,BMI[kg/㎡],24.12,22.59


In [170]:
# 将 `BMI[kg/㎡]` `腹囲[cm]` `空腹時血糖[mg/dl]` 转成 6 个 columns

temp1 = health_df[health_df.検査項目 == 'BMI[kg/㎡]'].reset_index()
temp1.rename(columns={'男性平均': 'BMI男性平均', '女性平均': 'BMI女性平均'}, inplace=True)

temp2 = health_df[health_df.検査項目 == '腹囲[cm]'].reset_index()
temp2.rename(columns={'男性平均': '腹围男性平均', '女性平均': '腹围女性平均', '都道府県': '--'}, inplace=True)

temp3 = health_df[health_df.検査項目 == '空腹時血糖[mg/dl]'].reset_index()
temp3.rename(columns={'男性平均': '血糖男性平均', '女性平均': '血糖女性平均', '都道府県': '--'}, inplace=True)

health_df_trans = pd.concat([temp1, temp2, temp3], axis=1)
health_df_trans

,index,都道府県,検査項目,BMI男性平均,BMI女性平均,index,--,検査項目,腹围男性平均,腹围女性平均,index,--,検査項目,血糖男性平均,血糖女性平均
0,0,北海道,BMI[kg/㎡],24.28,22.47,1,北海道,腹囲[cm],85.24,79.04,2,北海道,空腹時血糖[mg/dl],100.96,92.88
1,3,青森県,BMI[kg/㎡],24.13,22.83,4,青森県,腹囲[cm],84.43,79.94,5,青森県,空腹時血糖[mg/dl],104.20,95.67
2,6,岩手県,BMI[kg/㎡],24.07,22.80,7,岩手県,腹囲[cm],84.73,80.70,8,岩手県,空腹時血糖[mg/dl],102.76,94.59
3,9,宮城県,BMI[kg/㎡],24.12,22.59,10,宮城県,腹囲[cm],85.46,80.59,11,宮城県,空腹時血糖[mg/dl],101.87,93.56
4,12,秋田県,BMI[kg/㎡],24.09,22.76,13,秋田県,腹囲[cm],85.07,80.27,14,秋田県,空腹時血糖[mg/dl],103.78,95.41
5,15,山形県,BMI[kg/㎡],23.84,22.58,16,山形県,腹囲[cm],84.39,79.79,17,山形県,空腹時血糖[mg/dl],100.91,93.74
6,18,福島県,BMI[kg/㎡],24.10,22.82,19,福島県,腹囲[cm],85.25,80.72,20,福島県,空腹時血糖[mg/dl],102.69,95.57
7,21,茨城県,BMI[kg/㎡],24.04,22.53,22,茨城県,腹囲[cm],85.05,80.09,23,茨城県,空腹時血糖[mg/dl],102.88,94.77
8,24,栃木県,BMI[kg/㎡],23.98,22.53,25,栃木県,腹囲[cm],85.04,80.05,26,栃木県,空腹時血糖[mg/dl],100.38,93.86
9,27,群馬県,BMI[kg/㎡],23.83,22.46,28,群馬県,腹囲[cm],84.74,79.92,29,群馬県,空腹時血糖[mg/dl],100.97,94.47


In [172]:
# 保留有用的列

columns = ['都道府県', 'BMI男性平均', 'BMI女性平均', '腹围男性平均', '腹围女性平均', '血糖男性平均', '血糖女性平均']
health_df_trans = health_df_trans[columns]
health_df_trans

,都道府県,BMI男性平均,BMI女性平均,腹围男性平均,腹围女性平均,血糖男性平均,血糖女性平均
0,北海道,24.28,22.47,85.24,79.04,100.96,92.88
1,青森県,24.13,22.83,84.43,79.94,104.20,95.67
2,岩手県,24.07,22.80,84.73,80.70,102.76,94.59
3,宮城県,24.12,22.59,85.46,80.59,101.87,93.56
4,秋田県,24.09,22.76,85.07,80.27,103.78,95.41
5,山形県,23.84,22.58,84.39,79.79,100.91,93.74
6,福島県,24.10,22.82,85.25,80.72,102.69,95.57
7,茨城県,24.04,22.53,85.05,80.09,102.88,94.77
8,栃木県,23.98,22.53,85.04,80.05,100.38,93.86
9,群馬県,23.83,22.46,84.74,79.92,100.97,94.47


In [158]:
print(health_df_trans.to_csv(), file=open('人均烟税.csv', 'w'))